In [ ]:
# Dataset location (edit as needed)
root_dir = 'D:\\Spring 2022\\Deep Learning\\project\\SN7_buildings_train'

In [3]:
import multiprocessing
import pandas as pd
import numpy as np
import skimage
from osgeo import gdal
import sys
import os

import matplotlib as mpl
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
import matplotlib.colors as colors
plt.rcParams.update({'font.size': 16})
mpl.rcParams['figure.dpi'] = 300

import solaris as sol
from solaris.raster.image import create_multiband_geotiff
from solaris.utils.core import _check_gdf_load

# import from data_prep_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
# from sn7_baseline_prep_funcs import map_wrapper, make_geojsons_and_masks

In [4]:
# Make dataframe csvs for train/test

out_dir = os.path.join(root_dir, 'csvs/')
pops = ['train', 'test_public']
os.makedirs(out_dir, exist_ok=True)

for pop in pops: 
    d = os.path.join(root_dir, pop)
    outpath = os.path.join(out_dir, 'sn7_baseline_' + pop + '_df.csv')
    im_list, mask_list = [], []
    subdirs = sorted([f for f in os.listdir(d) if os.path.isdir(os.path.join(d, f))])
    for subdir in subdirs:
        
        if pop == 'train':
            im_files = [os.path.join(d, subdir, 'images_masked', f)
                    for f in sorted(os.listdir(os.path.join(d, subdir, 'images_masked')))
                    if f.endswith('.tif') and os.path.exists(os.path.join(d, subdir, 'masks', f.split('.')[0] + '_Buildings.tif'))]
            mask_files = [os.path.join(d, subdir, 'masks', f.split('.')[0] + '_Buildings.tif')
                      for f in sorted(os.listdir(os.path.join(d, subdir, 'images_masked')))
                      if f.endswith('.tif') and os.path.exists(os.path.join(d, subdir, 'masks', f.split('.')[0] + '_Buildings.tif'))]
            im_list.extend(im_files)
            mask_list.extend(mask_files)
    
        elif pop == 'test_public':
            im_files = [os.path.join(d, subdir, 'images_masked', f)
                    for f in sorted(os.listdir(os.path.join(d, subdir, 'images_masked')))
                    if f.endswith('.png')]
            im_list.extend(im_files)

    # save to dataframes
    # print("im_list:", im_list)
    # print("mask_list:", mask_list)
    if pop == 'train':
        df = pd.DataFrame({'image': im_list, 'label': mask_list})
        display(df.head())
    elif pop == 'test_public':
        df = pd.DataFrame({'image': im_list})
    df.to_csv(outpath, index=False)
    print(pop, "len df:", len(df))
    print("output csv:", outpath)

,image,label
0,D:\Spring 2022\Deep Learning\project\SN7_build...,D:\Spring 2022\Deep Learning\project\SN7_build...
1,D:\Spring 2022\Deep Learning\project\SN7_build...,D:\Spring 2022\Deep Learning\project\SN7_build...
2,D:\Spring 2022\Deep Learning\project\SN7_build...,D:\Spring 2022\Deep Learning\project\SN7_build...
3,D:\Spring 2022\Deep Learning\project\SN7_build...,D:\Spring 2022\Deep Learning\project\SN7_build...
4,D:\Spring 2022\Deep Learning\project\SN7_build...,D:\Spring 2022\Deep Learning\project\SN7_build...


train len df: 1260
output csv: D:\Spring 2022\Deep Learning\project\SN7_buildings_train\csvs/sn7_baseline_train_df.csv
test_public len df: 19
output csv: D:\Spring 2022\Deep Learning\project\SN7_buildings_train\csvs/sn7_baseline_test_public_df.csv


In [5]:
import glob 
import cv2
from matplotlib import pyplot as plt

In [6]:
data=[]
count=0

for p in glob.glob('D:\\Spring 2022\\Deep Learning\\project\\SN7_buildings_train\\test_public\\dha phase 2\\images_masked\*' +'.jpg'):
    img = cv2.imread(p)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # plt.imshow(img)
    # print(img.shape)
    
    resized = cv2.resize(img, (512,512), interpolation = cv2.INTER_AREA)
    plt.imsave("D:\\Spring 2022\\Deep Learning\\project\\SN7_buildings_train\\test_public\\dha phase 2\\images_masked\\img_resized_"+str(count)+'.jpg', resized)
    count+=1
    # img.resize(512,512,3)
    
    data.append(resized)
# plt.imshow(data[3])

In [1]:
import solaris as sol
import os
config_path = '../yml/sn7_baseline_infer.yml'
config = sol.utils.config.parse(config_path)
print('Config:')
print(config)

# make infernce output dir
os.makedirs(os.path.dirname(config['inference']['output_dir']), exist_ok=True)
inferer = sol.nets.infer.Inferer(config)
inferer()

c:\Users\Syed Talal\anaconda3\envs\solaris\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\Syed Talal\anaconda3\envs\solaris\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\Syed Talal\anaconda3\envs\solaris\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\Syed Talal\anaconda3\envs\solaris\lib\site-

Config:
{'model_name': 'xdxd_spacenet4', 'model_path': 'D:\\\\Spring 2022\\\\Deep Learning\\\\project\\\\trained_weights.pth', 'train': False, 'infer': True, 'pretrained': True, 'nn_framework': 'torch', 'batch_size': 6, 'data_specs': {'width': 512, 'height': 512, 'dtype': None, 'image_type': 'zscore', 'rescale': False, 'rescale_minima': 'auto', 'rescale_maxima': 'auto', 'channels': 4, 'label_type': 'mask', 'is_categorical': False, 'mask_channels': 1, 'val_holdout_frac': 0.1, 'data_workers': None}, 'training_data_csv': None, 'validation_data_csv': None, 'inference_data_csv': 'D:\\\\Spring 2022\\\\Deep Learning\\\\project\\\\SN7_buildings_train\\\\csvs\\\\sn7_baseline_test_public_df.csv', 'training_augmentation': {'augmentations': {'DropChannel': {'idx': 3, 'axis': 2}, 'HorizontalFlip': {'p': 0.5}, 'RandomRotate90': {'p': 0.5}, 'RandomCrop': {'height': 512, 'width': 512, 'p': 1.0}}, 'p': 1.0, 'shuffle': True}, 'validation_augmentation': {'augmentations': {'DropChannel': {'idx': 3, 'axis'